In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [22]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import xgboost as xgb

In [23]:

from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import jaccard_score
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

In [24]:
df_train = pd.read_csv(r'C:\Users\100_rabh\1. protium\Train_v1.2.csv',sep=',',encoding='utf-8',parse_dates=True)
df_test = pd.read_csv(r'C:\Users\100_rabh\1. protium\Test_v1.2.csv',sep=',',encoding='utf-8',parse_dates=True)

In [25]:
print("Shape of train:", df_train.shape)
print("Shape of test:", df_test.shape)

Shape of train: (97480, 34)
Shape of test: (24370, 34)


In [26]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97480 entries, 0 to 97479
Data columns (total 34 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Patron_Salary                               97480 non-null  float64
 1   Automobile_Possession                       97480 non-null  float64
 2   Two-Wheeler_Ownership                       97480 non-null  float64
 3   Ongoing_Borrowing                           97480 non-null  float64
 4   Residence_Proprietorship                    97480 non-null  float64
 5   Offspring_Number                            97480 non-null  float64
 6   Loan_Capital                                97480 non-null  float64
 7   Borrowing_Periodic_Payment                  97480 non-null  float64
 8   Customer_Revenue_Category                   97480 non-null  object 
 9   Patron_Academic_Qualification               97480 non-null  object 
 10  Customer_C

In [27]:
for i in df_train.columns:
    if df_train[i].dtypes == 'object':
        df_train[i] = df_train[i].str.upper()
        df_train[i] = df_train[i].str.strip()
        
        df_test[i] = df_test[i].str.upper()
        df_test[i] = df_test[i].str.strip()

In [28]:
# converting float to integer if column have 2 unique value
for i in df_train.columns:
    if df_train[i].dtypes == 'float':
        if df_train[i].nunique() <= 2:
            df_train[i] = df_train[i].astype(int)

In [29]:
# converting float to integer if column have 2 unique value
for i in df_test.columns:
    if df_test[i].dtypes == 'float':
        if df_test[i].nunique() <= 2:
            df_test[i] = df_test[i].astype(int)

### IV

In [30]:
def iv_woe(data, target, bins, show_woe=False):
    
    #Empty Dataframe
    newDF,woeDF = pd.DataFrame(), pd.DataFrame()
    
    #Extract Column Names
    cols = data.columns
    
    #Run WOE and IV on all the independent variables
    for ivars in cols[~cols.isin([target])]:
        if (data[ivars].dtype.kind in 'bifc') and (len(np.unique(data[ivars]))>10):
            binned_x = pd.qcut(data[ivars], bins,  duplicates='drop')
            d0 = pd.DataFrame({'x': binned_x, 'y': data[target]})
        else:
            d0 = pd.DataFrame({'x': data[ivars], 'y': data[target]})
        d = d0.groupby("x", as_index=False).agg({"y": ["count", "sum"]})
        d.columns = ['Cutoff', 'N', 'Events']
        d['% of Events'] = np.maximum(d['Events'], 0.5) / d['Events'].sum()
        d['Non-Events'] = d['N'] - d['Events']
        d['% of Non-Events'] = np.maximum(d['Non-Events'], 0.5) / d['Non-Events'].sum()
        d['WoE'] = np.log(d['% of Events']/d['% of Non-Events'])
        d['IV'] = d['WoE'] * (d['% of Events'] - d['% of Non-Events'])
        d.insert(loc=0, column='Variable', value=ivars)
        print("Information value of " + ivars + " is " + str(round(d['IV'].sum(),6)))
        temp =pd.DataFrame({"Variable" : [ivars], "IV" : [d['IV'].sum()]}, columns = ["Variable", "IV"])
        newDF=pd.concat([newDF,temp], axis=0)
        woeDF=pd.concat([woeDF,d], axis=0)

        #Show WOE Table
        if show_woe == True:
            print(d)
    return newDF, woeDF

In [31]:
iv, woe = iv_woe(data = df_train, target = 'Default', bins = 4, show_woe = True)

Information value of Patron_Salary is 0.010128
        Variable               Cutoff      N  Events  % of Events  Non-Events  \
0  Patron_Salary  (6749.999, 11250.0]  30980    2570     0.326349       28410   
1  Patron_Salary   (11250.0, 14400.0]  18366    1596     0.202667       16770   
2  Patron_Salary   (14400.0, 20250.0]  26138    2223     0.282286       23915   
3  Patron_Salary   (20250.0, 33750.0]  21996    1486     0.188698       20510   

   % of Non-Events       WoE        IV  
0         0.317058  0.028883  0.000268  
1         0.187155  0.079627  0.001235  
2         0.266894  0.056070  0.000863  
3         0.228893 -0.193107  0.007762  
Information value of Automobile_Possession is 0.007117
                Variable  Cutoff      N  Events  % of Events  Non-Events  \
0  Automobile_Possession       0  65025    5537     0.703111       59488   
1  Automobile_Possession       1  32455    2338     0.296889       30117   

   % of Non-Events       WoE        IV  
0         0.66389

Information value of Enlistment_Period_in_Days is 0.019159
                    Variable              Cutoff      N  Events  % of Events  \
0  Enlistment_Period_in_Days  (343.999, 2087.75]  24370    2187     0.277714   
1  Enlistment_Period_in_Days   (2087.75, 4406.0]  24376    2123     0.269587   
2  Enlistment_Period_in_Days   (4406.0, 7352.25]  24364    2001     0.254095   
3  Enlistment_Period_in_Days  (7352.25, 11339.0]  24370    1564     0.198603   

   Non-Events  % of Non-Events       WoE        IV  
0       22183         0.247564  0.114922  0.003465  
1       22253         0.248346  0.082071  0.001743  
2       22363         0.249573  0.017957  0.000081  
3       22806         0.254517 -0.248059  0.013870  
Information value of Identity_Age_in_Days is 0.04018
               Variable             Cutoff      N  Events  % of Events  \
0  Identity_Age_in_Days  (383.999, 1789.0]  24373    2442     0.310095   
1  Identity_Age_in_Days   (1789.0, 3148.0]  24377    2093     0.265778   


Information value of Rating_Origin_2 is 0.227251
          Variable          Cutoff      N  Events  % of Events  Non-Events  \
0  Rating_Origin_2  (0.137, 0.403]  24370    3341     0.424254       21029   
1  Rating_Origin_2  (0.403, 0.561]  24371    1960     0.248889       22411   
2  Rating_Origin_2   (0.561, 0.66]  24369    1605     0.203810       22764   
3  Rating_Origin_2   (0.66, 0.746]  24370     969     0.123048       23401   

   % of Non-Events       WoE        IV  
0         0.234686  0.592086  0.112241  
1         0.250109 -0.004889  0.000006  
2         0.254048 -0.220339  0.011070  
3         0.261157 -0.752551  0.103935  
Information value of Rating_Origin_3 is 0.246068
          Variable          Cutoff      N  Events  % of Events  Non-Events  \
0  Rating_Origin_3  (0.179, 0.424]  24458    3359     0.426540       21099   
1  Rating_Origin_3  (0.424, 0.521]  28208    2390     0.303492       25818   
2  Rating_Origin_3  (0.521, 0.631]  20631    1178     0.149587       194

In [32]:
# iv.to_excel('IV.xlsx', index = False)

### VIF

In [33]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calc_vif(df1):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = df1.columns
    vif["VIF"] = [float(format(variance_inflation_factor(df1.values, i),'f') )for i in range(df1.shape[1])]

    return(vif)

In [34]:
df2 = df_train._get_numeric_data()

In [35]:
# VIF=calc_vif(df2)

In [40]:
# VIF.sort_values('VIF',ascending=False)

In [41]:
# VIF.to_excel('VIF.xlsx', index = False)

### Information Gain

In [42]:
def calc_entropy(column):
    """
    Calculate entropy given a series, list, or numpy array.
    """
    # Compute the counts of each given value in the column
    counts = np.bincount(column)
    # Divide by the total column length to get a probability
    probabilities = counts / len(column)
    
    # Initialize the entropy to 0
    entropy = 0
    # Loop through the probabilities, and add each one to the total entropy
    for prob in probabilities:
        if prob > 0:
            # use log from math and set base to 2
            entropy += prob * math.log(prob, 2)
    
    return -entropy

In [43]:
def calc_information_gain(data, split_name, target_name):
    """
    Calculate information gain given a data set, column to split on and target.
    """
    # Calculate the original entropy
    original_entropy = calc_entropy(data[target_name])
    
    # Find the unique values in the column
    values = data[split_name].unique()
    
    print(split_name, ':', values)
    left_split = (data[data[split_name] == values[0]])
    right_split = (data[data[split_name] == values[1]])
    
    # Loop through the splits and calculate the subset entropies
    to_subtract = 0
    for subset in [left_split, right_split]:
        prob = (subset.shape[0] / data.shape[0])
        to_subtract += prob * calc_entropy(subset[target_name])
        
    # Return information gain
    return original_entropy - to_subtract

In [44]:
inf_gain = []

for i in df_train.columns:
    try:
        inf = calc_information_gain(df_train, i, 'Default')
    except:
        inf = "error"
        
    inf_gain.append({
        'VARIABLE': i,
        'Information Gain': inf
    })

df_inf_gain = pd.DataFrame(inf_gain)
df_inf_gain.to_excel('IG.xlsx', index = False)

In [46]:
model_var = [
    'Patron_Salary',
    'Automobile_Possession',
    'Offspring_Number',
    'Loan_Capital',
    'Patron_Academic_Qualification',
    'Patron_Sex',
    'Borrowing_Agreement_Category',
    'Customer_Living_Arrangement',
    'Elderliness_in_Days',
    'Work_Duration_in_Days',
    'Identity_Age_in_Days',
    'Employment_Phone_Operation',
    'Patron_Constant_Correspondence_Marker',
    'Rating_Origin_2',
    'Telecommunication_Switch',
]

### WOE Binning

In [23]:
num_biv = pd.read_excel(r'C:\Users\100_rabh\1. protium\model\bivariate\numerical_bi_variate.xlsx')

In [24]:
num_biv.head()

,variable,bin,good,bad,count,percentage
0,Patron_Salary,"[-inf,22500.0)",70132,6491,76623,0.085
1,Patron_Salary,"[22500.0,26500.0)",8468,665,9133,0.073
2,Patron_Salary,"[26500.0,inf)",11007,722,11729,0.062
3,Automobile_Possession,"[-inf,1.0)",59488,5539,65027,0.085
4,Automobile_Possession,"[1.0,inf)",30119,2339,32458,0.072


In [25]:
def find(text):
    str1 = text

    return (str1.translate({ord(i): None for i in '%'}))

In [26]:
num_biv['bin'] = num_biv.apply(lambda x: find(x['bin']), axis=1)

### NUMERICAL VARIABLE BINNING

In [27]:
def num_label(row, col, conditions):
    if len(conditions) == 2:
        if (row[col] < conditions[0]):
            return 'Group0'
        elif (conditions[0] <= row[col]) & (row[col] < conditions[1]):
            return 'Group1'
        else:
            return 'Group2'
        
    elif len(conditions) == 3:
        if (row[col] < conditions[0]):
            return 'Group0'
        elif (conditions[0] <= row[col]) & (row[col] < conditions[1]):
            return 'Group1'
        elif (conditions[1] <= row[col]) & (row[col] < conditions[2]):
            return 'Group2'
        else:
            return 'Group3'
        
    elif len(conditions) == 4:
        if (row[col] < conditions[0]):
            return 'Group0'
        elif (conditions[0] <= row[col]) & (row[col] < conditions[1]):
            return 'Group1'
        elif (conditions[1] <= row[col]) & (row[col] < conditions[2]):
            return 'Group2'
        elif (conditions[2] <= row[col]) & (row[col] < conditions[3]):
            return 'Group3'
        else:
            return'Group4'
    
    elif len(conditions) == 5:
        if (row[col] < conditions[0]):
            return 'Group0'
        elif (conditions[0] <= row[col]) & (row[col] < conditions[1]):
            return 'Group1'
        elif (conditions[1] <= row[col]) & (row[col] < conditions[2]):
            return 'Group2'
        elif (conditions[2] <= row[col]) & (row[col] < conditions[3]):
            return 'Group3'
        elif (conditions[3] <= row[col]) & (row[col] < conditions[4]):
            return'Group4'
        else:
            return 'Group5'
    
    else:
        return 'NONE'
        
def catg_bin_num(row, col, num_biv):
    num_biv_1 = num_biv[num_biv['variable'] == col].reset_index(drop = True)
    conditions = []
    for i in num_biv_1.index:
        conditions.append(float(num_biv_1['bin'][i][1:-1].split(',')[1]))
    conditions = conditions[:-1]
    print(conditions)
    row[col + str('_BIN')] = row.apply(lambda row: num_label(row, col, conditions), axis=1)
    return row

In [28]:
df_train = catg_bin_num(df_train, 'Patron_Salary', num_biv)
df_test = catg_bin_num(df_test, 'Patron_Salary', num_biv)

[22500.0, 26500.0]
[22500.0, 26500.0]


In [29]:
df_train['Patron_Salary_BIN'].value_counts()

Group0    76618
Group2    11729
Group1     9133
Name: Patron_Salary_BIN, dtype: int64

In [30]:
df_train = catg_bin_num(df_train, 'Loan_Capital', num_biv)
df_test = catg_bin_num(df_test, 'Loan_Capital', num_biv)

[66000.0, 90000.0, 108000.0]
[66000.0, 90000.0, 108000.0]


In [31]:
df_train['Loan_Capital_BIN'].value_counts()

Group0    62404
Group1    15042
Group3    11781
Group2     8253
Name: Loan_Capital_BIN, dtype: int64

In [32]:
df_train = catg_bin_num(df_train, 'Elderliness_in_Days', num_biv)
df_test = catg_bin_num(df_test, 'Elderliness_in_Days', num_biv)

[12600.0, 14800.0, 20200.0]
[12600.0, 14800.0, 20200.0]


In [33]:
df_train['Elderliness_in_Days_BIN'].value_counts()

Group2    33732
Group0    24987
Group3    20835
Group1    17926
Name: Elderliness_in_Days_BIN, dtype: int64

In [34]:
df_train = catg_bin_num(df_train, 'Work_Duration_in_Days', num_biv)
df_test = catg_bin_num(df_test, 'Work_Duration_in_Days', num_biv)

[1000.0, 2000.0, 4000.0]
[1000.0, 2000.0, 4000.0]


In [35]:
df_train['Work_Duration_in_Days_BIN'].value_counts()

Group3    30908
Group0    25241
Group1    21446
Group2    19885
Name: Work_Duration_in_Days_BIN, dtype: int64

In [36]:
df_train = catg_bin_num(df_train, 'Identity_Age_in_Days', num_biv)
df_test = catg_bin_num(df_test, 'Identity_Age_in_Days', num_biv)

[1600.0, 4100.0, 4600.0]
[1600.0, 4100.0, 4600.0]


In [37]:
df_train['Identity_Age_in_Days_BIN'].value_counts()

Group1    44491
Group0    21721
Group2    18522
Group3    12746
Name: Identity_Age_in_Days_BIN, dtype: int64

In [38]:
df_train = catg_bin_num(df_train, 'Identity_Age_in_Days', num_biv)
df_test = catg_bin_num(df_test, 'Identity_Age_in_Days', num_biv)

[1600.0, 4100.0, 4600.0]
[1600.0, 4100.0, 4600.0]


In [39]:
df_train['Identity_Age_in_Days_BIN'].value_counts()

Group1    44491
Group0    21721
Group2    18522
Group3    12746
Name: Identity_Age_in_Days_BIN, dtype: int64

In [40]:
df_train = catg_bin_num(df_train, 'Rating_Origin_2', num_biv)
df_test = catg_bin_num(df_test, 'Rating_Origin_2', num_biv)

[0.23, 0.4600000000000002, 0.6700000000000004, 0.7200000000000004]
[0.23, 0.4600000000000002, 0.6700000000000004, 0.7200000000000004]


In [41]:
df_train['Rating_Origin_2_BIN'].value_counts()

Group2    45488
Group1    20036
Group3    11919
Group0    10269
Group4     9768
Name: Rating_Origin_2_BIN, dtype: int64

In [42]:
df_train = catg_bin_num(df_train, 'Telecommunication_Switch', num_biv)
df_test = catg_bin_num(df_test, 'Telecommunication_Switch', num_biv)

[1050.0, 1950.0]
[1050.0, 1950.0]


In [43]:
df_train['Telecommunication_Switch_BIN'].value_counts()

Group0    59528
Group1    25116
Group2    12836
Name: Telecommunication_Switch_BIN, dtype: int64

### CATEGORICAL VARIABLE BINNING 

In [44]:
def label(row, col, conditions):
    for i in range(len(conditions)):
        if row[col] in conditions[i]:
            return 'Group' + str(i)
        
    
def catg_bin(row, col, num_biv):
    num_biv_1 = num_biv[num_biv['variable'] == col].reset_index(drop = True)
    conditions = []
    for i in num_biv_1.index:
        conditions.append(num_biv_1['bin'][i].split(','))
    
    print(conditions)
    row[col + str('_BIN')] = row.apply(lambda row: label(row, col, conditions), axis=1)
    return row

In [45]:
df_train = catg_bin(df_train, 'Patron_Academic_Qualification', num_biv)
df_test = catg_bin(df_test, 'Patron_Academic_Qualification', num_biv)

[['POST GRAD', 'GRADUATION'], ['GRADUATION DROPOUT', 'SECONDARY', 'JUNIOR SECONDARY']]
[['POST GRAD', 'GRADUATION'], ['GRADUATION DROPOUT', 'SECONDARY', 'JUNIOR SECONDARY']]


In [46]:
df_train['Patron_Academic_Qualification_BIN'].value_counts()

Group1    74338
Group0    23142
Name: Patron_Academic_Qualification_BIN, dtype: int64

In [47]:
df_train['Patron_Academic_Qualification_BIN'].replace({'Group0': 0, 'Group1': 1}, inplace = True)
df_test['Patron_Academic_Qualification_BIN'].replace({'Group0': 0, 'Group1': 1}, inplace = True)

In [48]:
df_train['Patron_Academic_Qualification_BIN'].value_counts()

1    74338
0    23142
Name: Patron_Academic_Qualification_BIN, dtype: int64

In [49]:
df_test['Patron_Academic_Qualification'].value_counts()

SECONDARY             17524
GRADUATION             5727
GRADUATION DROPOUT      799
JUNIOR SECONDARY        304
POST GRAD                16
Name: Patron_Academic_Qualification, dtype: int64

In [50]:
df_test['Patron_Academic_Qualification_BIN'].value_counts()

1    18627
0     5743
Name: Patron_Academic_Qualification_BIN, dtype: int64

In [51]:
df_train = catg_bin(df_train, 'Patron_Sex', num_biv)
df_test = catg_bin(df_test, 'Patron_Sex', num_biv)

[['MALE'], ['FEMALE']]
[['MALE'], ['FEMALE']]


In [52]:
df_train['Patron_Sex_BIN'].value_counts()

Group0    64646
Group1    32834
Name: Patron_Sex_BIN, dtype: int64

In [53]:
df_train['Patron_Sex_BIN'].replace({'Group0': 0, 'Group1': 1}, inplace = True)
df_test['Patron_Sex_BIN'].replace({'Group0': 0, 'Group1': 1}, inplace = True)

In [54]:
df_train['Patron_Sex_BIN'].value_counts()

0    64646
1    32834
Name: Patron_Sex_BIN, dtype: int64

In [55]:
df_train = catg_bin(df_train, 'Borrowing_Agreement_Category', num_biv)
df_test = catg_bin(df_test, 'Borrowing_Agreement_Category', num_biv)

[['RL'], ['CL']]
[['RL'], ['CL']]


In [56]:
df_train['Borrowing_Agreement_Category_BIN'].value_counts()

Group1    88654
Group0     8826
Name: Borrowing_Agreement_Category_BIN, dtype: int64

In [57]:
df_train['Borrowing_Agreement_Category_BIN'].replace({'Group0': 0, 'Group1': 1}, inplace = True)
df_test['Borrowing_Agreement_Category_BIN'].replace({'Group0': 0, 'Group1': 1}, inplace = True)

In [58]:
df_train['Borrowing_Agreement_Category_BIN'].value_counts()

1    88654
0     8826
Name: Borrowing_Agreement_Category_BIN, dtype: int64

In [59]:
df_train = catg_bin(df_train, 'Customer_Living_Arrangement', num_biv)
df_test = catg_bin(df_test, 'Customer_Living_Arrangement', num_biv)

[['OFFICE', 'HOME', 'MUNICIPAL'], ['SHARED', 'FAMILY', 'RENTAL']]
[['OFFICE', 'HOME', 'MUNICIPAL'], ['SHARED', 'FAMILY', 'RENTAL']]


In [60]:
df_train['Customer_Living_Arrangement_BIN'].value_counts()

Group0    91049
Group1     6431
Name: Customer_Living_Arrangement_BIN, dtype: int64

In [61]:
df_train['Customer_Living_Arrangement_BIN'].replace({'Group0': 0, 'Group1': 1}, inplace = True)
df_test['Customer_Living_Arrangement_BIN'].replace({'Group0': 0, 'Group1': 1}, inplace = True)


In [62]:
df_train['Customer_Living_Arrangement_BIN'].value_counts()

0    91049
1     6431
Name: Customer_Living_Arrangement_BIN, dtype: int64

In [63]:
df_train = catg_bin(df_train, 'Patron_Constant_Correspondence_Marker', num_biv)
df_test = catg_bin(df_test, 'Patron_Constant_Correspondence_Marker', num_biv)

[['YES'], ['NO']]
[['YES'], ['NO']]


In [64]:
df_train['Patron_Constant_Correspondence_Marker_BIN'].value_counts()

Group0    89933
Group1     7547
Name: Patron_Constant_Correspondence_Marker_BIN, dtype: int64

In [65]:
df_train['Patron_Constant_Correspondence_Marker_BIN'].replace({'Group0': 0, 'Group1': 1}, inplace = True)
df_test['Patron_Constant_Correspondence_Marker_BIN'].replace({'Group0': 0, 'Group1': 1}, inplace = True)


In [66]:
df_train['Patron_Constant_Correspondence_Marker_BIN'].value_counts()

0    89933
1     7547
Name: Patron_Constant_Correspondence_Marker_BIN, dtype: int64

In [67]:
train = df_train.copy()
test = df_test.copy()

### Model Development

In [134]:
df_train = train.copy()
df_test = test.copy()

In [135]:
model_var = [
#     'Patron_Salary_BIN',
    'Patron_Salary',
    'Automobile_Possession',
#     'Offspring_Number',
    'Loan_Capital',
#     'Loan_Capital_BIN',
    'Patron_Academic_Qualification_BIN',
#     'Patron_Sex_BIN',
    'Borrowing_Agreement_Category_BIN',
    'Customer_Living_Arrangement_BIN',
    'Elderliness_in_Days',
#     'Elderliness_in_Days_BIN',
    'Work_Duration_in_Days',
#     'Work_Duration_in_Days_BIN',
    'Identity_Age_in_Days',
#     'Identity_Age_in_Days_BIN',
    'Employment_Phone_Operation',
    'Patron_Constant_Correspondence_Marker_BIN',
    'Rating_Origin_2',
#     'Rating_Origin_2_BIN',
    'Telecommunication_Switch',
#     'Telecommunication_Switch_BIN',
    'Rating_Origin_3',
]

In [136]:
df_train.columns

Index(['Patron_Salary', 'Automobile_Possession', 'Two-Wheeler_Ownership',
       'Ongoing_Borrowing', 'Residence_Proprietorship', 'Offspring_Number',
       'Loan_Capital', 'Borrowing_Periodic_Payment',
       'Customer_Revenue_Category', 'Patron_Academic_Qualification',
       'Customer_Conjugal_State', 'Patron_Sex', 'Borrowing_Agreement_Category',
       'Customer_Living_Arrangement', 'Population_Fraction_by_Region',
       'Elderliness_in_Days', 'Work_Duration_in_Days',
       'Enlistment_Period_in_Days', 'Identity_Age_in_Days', 'Cellphone_Marker',
       'Residential_Phone_Marker', 'Employment_Phone_Operation',
       'Patron_Kin_Count', 'Customer_Urban_Area_Ranking',
       'Request_Submission_Day', 'Request_Submission_Hour',
       'Patron_Constant_Correspondence_Marker',
       'Customer_Professional_Communication_Marker', 'Sort_of_Institution',
       'Rating_Origin_2', 'Rating_Origin_3', 'Telecommunication_Switch',
       'Solvency_Information_Agency', 'Default', 'Patron_Salar

In [137]:
df_train1 = df_train[model_var]

cat_vars = [i for i in df_train1.columns if df_train1[i].dtype == 'object']

for var in cat_vars:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(df_train1[var], prefix=var)
    df_train1=df_train1.join(cat_list)
    
df_train1=df_train1.drop(cat_vars,axis=1)

df_test1 = df_test[model_var]
for var in cat_vars:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(df_test1[var], prefix=var)
    df_test1=df_test1.join(cat_list)
    
df_test1=df_test1.drop(cat_vars,axis=1)

print("Training set : ",df_train1.shape)
print("Test set : ",df_test1.shape)

Training set :  (97480, 14)
Test set :  (24370, 14)


In [138]:
def maximum(a, b):
    if a != b:
        if a[0] >= b[0]:
            largest = a[1]
        else:
            largest = b[1]
            
        return largest
    else:
        print("Equal shapes")
        
tr1 = [len(df_train1.columns), "train"]
te1 = [len(df_test1.columns), "test"]

largest = maximum(tr1, te1)
print("Maximum shape of", largest, "dataframe!!!")

Maximum shape of train dataframe!!!


In [139]:
tr_col = set(df_train1.columns.to_list())
te_col = set(df_test1.columns.to_list())

if largest == 'train':
    x = tr_col.intersection(te_col)
    for i in list(tr_col):
        if i not in list(x):
            print("test:", i)
            df_test1[i] = 0

if largest == 'test':
    x = te_col.intersection(tr_col)
    for i in list(tr_col):
        if i not in list(x):
            print("train:", i)
            df_train1[i] = 0

else:
    pass

print("----------------------------------------")
print("Training set: ", df_train1.shape)
print("Testing set: ", df_test1.shape)

----------------------------------------
Training set:  (97480, 14)
Testing set:  (24370, 14)


In [140]:
X_train = df_train1.copy()
y_train = df_train['Default']
print("Shape of X_train: ", X_train.shape)
print("Shape of y_train: ", y_train.shape)
print("\n")

X_test = df_test1.copy()
y_test = df_test['Default']
print("Shape of X_test: ", X_test.shape)
print("Shape of y_test: ", y_test.shape)
print("\n")

Shape of X_train:  (97480, 14)
Shape of y_train:  (97480,)


Shape of X_test:  (24370, 14)
Shape of y_test:  (24370,)




In [144]:
f_imp = [
    'Rating_Origin_2', 'Work_Duration_in_Days',
       'Patron_Academic_Qualification_BIN', 'Elderliness_in_Days',
       'Telecommunication_Switch',
       'Customer_Living_Arrangement_BIN', 'Automobile_Possession',
       'Patron_Constant_Correspondence_Marker_BIN',
       'Identity_Age_in_Days', 'Loan_Capital', 'Rating_Origin_3',
#        'Borrowing_Agreement_Category_BIN', 'Patron_Salary',
#        'Employment_Phone_Operation'
]

In [145]:
X_train = X_train[f_imp]
X_test = X_test[f_imp]

In [146]:
X_train.shape

(97480, 11)

In [147]:
X_train.head(2)

,Rating_Origin_2,Work_Duration_in_Days,Patron_Academic_Qualification_BIN,Elderliness_in_Days,Telecommunication_Switch,Customer_Living_Arrangement_BIN,Automobile_Possession,Patron_Constant_Correspondence_Marker_BIN,Identity_Age_in_Days,Loan_Capital,Rating_Origin_3
0,0.205747,365243.0,1,20099.0,0.0,0,0,0,3514.0,101961.0,0.593718
1,0.679736,365243.0,1,22293.0,1046.0,0,0,0,4492.0,18201.6,0.600658


In [148]:
X_test.shape

(24370, 11)

In [149]:
X_test.head(2)

,Rating_Origin_2,Work_Duration_in_Days,Patron_Academic_Qualification_BIN,Elderliness_in_Days,Telecommunication_Switch,Customer_Living_Arrangement_BIN,Automobile_Possession,Patron_Constant_Correspondence_Marker_BIN,Identity_Age_in_Days,Loan_Capital,Rating_Origin_3
0,0.552795,5102.0,1,16790.0,277.0,1,0,0,393.0,59527.35,0.329655
1,0.418630,2504.0,1,14630.0,2458.0,0,0,0,1191.0,22500.00,0.595456


In [150]:
X_train.columns

Index(['Rating_Origin_2', 'Work_Duration_in_Days',
       'Patron_Academic_Qualification_BIN', 'Elderliness_in_Days',
       'Telecommunication_Switch', 'Customer_Living_Arrangement_BIN',
       'Automobile_Possession', 'Patron_Constant_Correspondence_Marker_BIN',
       'Identity_Age_in_Days', 'Loan_Capital', 'Rating_Origin_3'],
      dtype='object')

### XG Boost

In [151]:
import xgboost as xgb
xgb_cl = xgb.XGBClassifier(objective='binary:logistic', booster='gbtree',learning_rate=0.01,max_depth=7,
                               base_score=0.5, n_estimators=200, num_parallel_tree=6, eval_metric='auc',
                               tree_method='hist', grow_policy='lossguide', scale_pos_weight = 10,
                               gamma = 30, colsample_bytree = 0.5
                              ).fit(X_train, y_train)

y_pred_xg1 = xgb_cl.predict(X_train)
y_pred_xg = xgb_cl.predict(X_test)

X_train_prob = list(xgb_cl.predict_proba(X_train)[:,1])
X_test_prob = list(xgb_cl.predict_proba(X_test)[:,1])

train['Prob_score_XGB'] = X_train_prob
test['Prob_score_XGB'] = X_test_prob

In [152]:
print("ROC AUC Score for train:", roc_auc_score(y_train, X_train_prob) * 100)
print("ROC AUC Score for test:", roc_auc_score(y_test, X_test_prob) * 100)

ROC AUC Score for train: 75.8812624366377
ROC AUC Score for test: 73.80209151576834


In [153]:
feat_imp = pd.DataFrame.from_dict(zip(list(X_train.columns), xgb_cl.feature_importances_*100)).sort_values(by = 1, ascending = False)
# feat_imp.to_excel('feature_imp_xgb.xlsx', index = False)
feat_imp

,0,1
10,Rating_Origin_3,25.178265
0,Rating_Origin_2,22.332323
2,Patron_Academic_Qualification_BIN,16.748718
1,Work_Duration_in_Days,7.556044
3,Elderliness_in_Days,4.879126
4,Telecommunication_Switch,4.162874
9,Loan_Capital,4.127804
7,Patron_Constant_Correspondence_Marker_BIN,4.086364
8,Identity_Age_in_Days,3.925455
6,Automobile_Possession,3.832746


In [154]:
feat_imp[0].values

array(['Rating_Origin_3', 'Rating_Origin_2',
       'Patron_Academic_Qualification_BIN', 'Work_Duration_in_Days',
       'Elderliness_in_Days', 'Telecommunication_Switch', 'Loan_Capital',
       'Patron_Constant_Correspondence_Marker_BIN',
       'Identity_Age_in_Days', 'Automobile_Possession',
       'Customer_Living_Arrangement_BIN'], dtype=object)

In [155]:
def ks(data=None,target=None, prob=None):
    data['target_xgb'] = 1 - data[target]
    data['bucket'] = pd.qcut(data[prob], 10)
    grouped = data.groupby('bucket', as_index = False)
    kstable = pd.DataFrame()
    kstable['min_prob'] = grouped.min()[prob]
    kstable['max_prob'] = grouped.max()[prob]
    kstable['total'] = (grouped.sum()[target] + grouped.sum()['target_xgb'])
    kstable['events']   = grouped.sum()[target]
    kstable['nonevents'] = grouped.sum()['target_xgb']
    kstable = kstable.sort_values(by="min_prob", ascending=False).reset_index(drop = True)
    kstable['event_rate'] = (kstable.events / data[target].sum()).apply('{0:.2%}'.format)
    kstable['nonevent_rate'] = (kstable.nonevents / data['target_xgb'].sum()).apply('{0:.2%}'.format)
    kstable['cum_eventrate']=(kstable.events / data[target].sum()).cumsum()
    kstable['cum_noneventrate']=(kstable.nonevents / data['target_xgb'].sum()).cumsum()
    kstable['KS'] = np.round(kstable['cum_eventrate']-kstable['cum_noneventrate'], 3) * 100

    #Formating
    kstable['cum_eventrate']= kstable['cum_eventrate'].apply('{0:.2%}'.format)
    kstable['cum_noneventrate']= kstable['cum_noneventrate'].apply('{0:.2%}'.format)
    
    kstable['ks_stats'] = np.round(((kstable['events'] / kstable['events'].sum()).cumsum() -(kstable['nonevents'] / kstable['nonevents'].sum()).cumsum()), 4) * 100
    kstable['max_ks'] = kstable['ks_stats'].apply(lambda x: '*****' if x == kstable['ks_stats'].max() else '')
    
    kstable.index = range(1,11)
    kstable.index.rename('Decile', inplace=True)
    pd.set_option('display.max_columns', 20)
#    print(kstable)
#    print(type(kstable))    
    #Display KS
    from colorama import Fore
#   print(Fore.RED + "KS is " + str(max(kstable['KS']))+"%"+ " at decile " + str((kstable.index[kstable['KS']==max(kstable['KS'])][0])))
#     print(type(kstable))
    return(kstable)


In [156]:
xg_train_ks = ks(train,'Default','Prob_score_XGB')
xg_test_ks = ks(test,'Default','Prob_score_XGB')

In [157]:
# xg_train_ks.to_csv('xgboost_ks_train.csv', index = False)
xg_train_ks

,min_prob,max_prob,total,events,nonevents,event_rate,nonevent_rate,cum_eventrate,cum_noneventrate,KS,ks_stats,max_ks
Decile,,,,,,,,,,,,
1,0.596795,0.785409,9748,2566,7182,32.58%,8.02%,32.58%,8.02%,24.6,24.57,
2,0.534761,0.596793,9748,1472,8276,18.69%,9.24%,51.28%,17.25%,34.0,34.02,
3,0.487891,0.534748,9748,1056,8692,13.41%,9.70%,64.69%,26.95%,37.7,37.73,*****
4,0.448647,0.487885,9748,786,8962,9.98%,10.00%,74.67%,36.95%,37.7,37.71,
5,0.414545,0.448641,9748,650,9098,8.25%,10.15%,82.92%,47.11%,35.8,35.81,
6,0.383045,0.414544,9748,459,9289,5.83%,10.37%,88.75%,57.47%,31.3,31.28,
7,0.352345,0.383041,9748,363,9385,4.61%,10.47%,93.36%,67.95%,25.4,25.41,
8,0.319696,0.352343,9748,272,9476,3.45%,10.58%,96.81%,78.52%,18.3,18.29,
9,0.280606,0.319689,9748,162,9586,2.06%,10.70%,98.87%,89.22%,9.6,9.65,


In [158]:
# xg_test_ks.to_csv('xgboost_ks_test.csv', index = False)
xg_test_ks

,min_prob,max_prob,total,events,nonevents,event_rate,nonevent_rate,cum_eventrate,cum_noneventrate,KS,ks_stats,max_ks
Decile,,,,,,,,,,,,
1,0.595794,0.777810,2437,581,1856,29.54%,8.28%,29.54%,8.28%,21.3,21.25,
2,0.535189,0.595771,2437,364,2073,18.51%,9.25%,48.04%,17.54%,30.5,30.50,
3,0.488813,0.535177,2437,289,2148,14.69%,9.59%,62.74%,27.13%,35.6,35.61,
4,0.448875,0.488811,2437,203,2234,10.32%,9.97%,73.06%,37.10%,36.0,35.96,*****
5,0.414391,0.448871,2437,148,2289,7.52%,10.22%,80.58%,47.32%,33.3,33.26,
6,0.383272,0.414367,2437,131,2306,6.66%,10.29%,87.24%,57.61%,29.6,29.63,
7,0.352425,0.383266,2437,87,2350,4.42%,10.49%,91.66%,68.10%,23.6,23.56,
8,0.319611,0.352391,2437,60,2377,3.05%,10.61%,94.71%,78.71%,16.0,16.00,
9,0.281139,0.319606,2437,62,2375,3.15%,10.60%,97.86%,89.31%,8.6,8.56,


In [89]:


# Init classifier
def best_result(eta, md, npt, gam, train, test):
    xgb_cl = xgb.XGBClassifier(objective='binary:logistic', booster='gbtree',learning_rate=eta,max_depth=md,
                               base_score=0.4, n_estimators=200, num_parallel_tree=npt, eval_metric='auc',
                               tree_method='hist', grow_policy='lossguide', scale_pos_weight = 9,
                               gamma = gam, colsample_bytree = 0.5
                              ).fit(X_train, y_train)

    y_pred_xg1 = xgb_cl.predict(X_train)
    y_pred_xg = xgb_cl.predict(X_test)
    
    X_train_prob = list(xgb_cl.predict_proba(X_train)[:,1])
    X_test_prob = list(xgb_cl.predict_proba(X_test)[:,1])
#     print("max depth: " , md , ", num parallel tree: " , npt , "gamma: " , gam)
    train_res = roc_auc_score(y_train, X_train_prob) * 100
    test_res = roc_auc_score(y_test, X_test_prob) * 100
    
    train['Prob_score_XGB'] = X_train_prob
    test['Prob_score_XGB'] = X_test_prob

    xg_train_ks = ks(train,'Default','Prob_score_XGB')
    xg_test_ks = ks(test,'Default','Prob_score_XGB')
    
    train_3rd_cap = xg_train_ks.reset_index()['cum_eventrate'][2]
    test_3rd_cap = xg_test_ks.reset_index()['cum_eventrate'][2]
#     print(train_3rd_cap, test_3rd_cap)
#     print("ROC AUC Score for train:", train_res)
#     print("ROC AUC Score for test:", test_res)
    return eta, md, npt, gam, train_res, test_res, train_3rd_cap, test_3rd_cap

In [ ]:
my_dict = {"LearningRate": [], "Max_Depth":[], "Num_Parallel_Tree":[], "Gamma":[],
           "Train_AUC": [], "Test_AUC": [], "Train_3rd_DC": [], "Test_3rd_DC": []};
for i in [0.19, 0.195, 0.2, 0.21]:
    for j in [4, 5, 6, 7]:
        for k in [2, 4, 5, 6, 7]:
            for l in [70, 75, 80, 85, 90, 95]:
                try:
                    eta, md, npt, gam, train_res, test_res, train_3rd_cap, test_3rd_cap = best_result(i, j, k, l, train, test)
                    my_dict["LearningRate"].append(eta)
                    my_dict["Max_Depth"].append(md)
                    my_dict["Num_Parallel_Tree"].append(npt)
                    my_dict["Gamma"].append(gam)
                    my_dict["Train_AUC"].append(train_res)
                    my_dict["Test_AUC"].append(test_res)
                    my_dict["Train_3rd_DC"].append(train_3rd_cap)
                    my_dict["Test_3rd_DC"].append(test_3rd_cap)
#                     print(my_dict)
                except:
                    pass

                print("------------------------------------------------------------")


------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------

------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------

------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------

In [ ]:
my_dict

In [ ]:
# pd.DataFrame(my_dict).to_excel('output_v1.2.xlsx', index = False)

In [ ]:

# X_validation_prob = list(xgb_cl.predict_proba(X_validation)[:,1])


# validation['Prob_score_XGB'] = X_validation_prob


# print("ROC AUC Score for validation:", roc_auc_score(y_validation, X_validation_prob) * 100)


In [ ]:
feat_imp = pd.DataFrame.from_dict(zip(list(X_train.columns), xgb_cl.feature_importances_*100)).sort_values(by = 1, ascending = False)
# feat_imp.to_excel('feature_imp_xgb.xlsx', index = False)
feat_imp

In [ ]:
feat_imp[0].values

In [ ]:
# print()
print("ROC AUC Score for train:", roc_auc_score(y_train, y_pred_xg1) * 100)
print("ROC AUC Score for test:", roc_auc_score(y_test, y_pred_xg) * 100)
# print("ROC AUC Score for valid:", roc_auc_score(y_validation, y_pred_xg_v) * 100)

In [ ]:
# xg_train_ks.to_csv('xgboost_ks_train.csv', index = False)
xg_train_ks.reset_index()['cum_eventrate'][2]

In [ ]:
# xg_test_ks.to_csv('xgboost_ks_test.csv', index = False)
xg_test_ks

In [ ]:
# xg_validation_ks.to_csv('xgboost_ks_validation.csv', index = False)
# xg_validation_ks